In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import sklearn
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

## 1.Create recommender system using ratings.csv file. 

In [2]:
links=pd.read_csv('C:/Users/Administrator/Desktop/Ml_assignment/Assignment3/ml-latest-small/links.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [3]:
movies=pd.read_csv('C:/Users/Administrator/Desktop/Ml_assignment/Assignment3/ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
rating=pd.read_csv('C:/Users/Administrator/Desktop/Ml_assignment/Assignment3/ml-latest-small/ratings.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
rating.shape

(100836, 4)

In [6]:
tags=pd.read_csv('C:/Users/Administrator/Desktop/Ml_assignment/Assignment3/ml-latest-small/tags.csv')
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [7]:
tags.shape

(3683, 4)

In [8]:
rating.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [9]:
book_pivot=(rating.pivot(index='movieId',columns='userId',values='rating').fillna(0))


In [10]:
reader=Reader(rating_scale=(1,5))
data=Dataset.load_from_df(rating[['userId','movieId','rating']],reader)

In [11]:
[trainset,testset]=train_test_split(data,test_size=.15,shuffle=True)

In [12]:
recom=KNNWithMeans(k=50,sim_options={'name':'cosine','user_based':True})
recom.fit(data.build_full_trainset())
test_pred=recom.test(testset)
RMSE=accuracy.rmse(test_pred)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.7820


## 2.Display predicted rating for a particular userID and MovieID combinations (both taken as keyboard input)

In [13]:

pre=recom.predict(int(input('Enter the userID:')),int(input('Enter the MovieId:')))
print('The rating value for item 2 by user 2 is ',pre[3])

Enter the userID:5
Enter the MovieId:45
The rating value for item 2 by user 2 is  3.537975563130559


## 3.Recommend a user to watch top 10 movies, which has not watched yet (as per his rating predictions). Take userID as a keyboard input. Fix the threshold rating as 2.5

In [14]:
book_pivot1=book_pivot.T

In [15]:
ls_movies=book_pivot1.columns


In [53]:
dic={}
a=int(input())
for i in ls_movies:
    if book_pivot1[i][a]==0:
        p_rating=recom.predict(a,i)[3]
        dic[i]=p_rating

600


In [17]:
predicted_ratings=pd.Series(dic).sort_values(ascending=False)
predicted_ratings=predicted_ratings.reset_index()
predicted_ratings.columns=['movieId','Pred_rating']
predicted_ratings=predicted_ratings[predicted_ratings['Pred_rating']>=2.5]
predicted_ratings=predicted_ratings.head(10)
predicted_ratings

,movieId,Pred_rating
0,5746,5.0
1,5490,5.0
2,7899,5.0
3,8477,5.0
4,136850,5.0
5,40491,5.0
6,3567,5.0
7,156605,5.0
8,6835,5.0
9,132333,5.0


In [18]:
book_pivot1[132333][600]

0.0

## 4.	Display the MovieID, IMDB ID, Average User Rating (excluding predictions), genres and tag of all the movies found in Step 3 as a data frame.



In [19]:
ratings_mean_count = pd.DataFrame(rating.groupby('movieId')['rating'].mean())
ratings_mean_count['rating_counts'] = pd.DataFrame(rating.groupby('movieId')['rating'].count())
ratings_mean_count=ratings_mean_count.reset_index()
ratings_mean_count.head()


,movieId,rating,rating_counts
0,1,3.920930,215
1,2,3.431818,110
2,3,3.259615,52
3,4,2.357143,7
4,5,3.071429,49


In [20]:
movie_r=pd.merge(predicted_ratings,movies,on='movieId')
movie_r

,movieId,Pred_rating,title,genres
0,5746,5.0,Galaxy of Terror (Quest) (1981),Action|Horror|Mystery|Sci-Fi
1,5490,5.0,The Big Bus (1976),Action|Comedy
2,7899,5.0,Master of the Flying Guillotine (Du bi quan wa...,Action
3,8477,5.0,"Jetée, La (1962)",Romance|Sci-Fi
4,136850,5.0,Villain (1971),Crime|Drama|Thriller
5,40491,5.0,"Match Factory Girl, The (Tulitikkutehtaan tytt...",Comedy|Drama
6,3567,5.0,Bossa Nova (2000),Comedy|Drama|Romance
7,156605,5.0,Paterson,(no genres listed)
8,6835,5.0,Alien Contamination (1980),Action|Horror|Sci-Fi
9,132333,5.0,Seve (2014),Documentary|Drama


In [21]:
movie_r1=pd.merge(movie_r,ratings_mean_count,on='movieId')
movie_r1




,movieId,Pred_rating,title,genres,rating,rating_counts
0,5746,5.0,Galaxy of Terror (Quest) (1981),Action|Horror|Mystery|Sci-Fi,5.0,1
1,5490,5.0,The Big Bus (1976),Action|Comedy,5.0,1
2,7899,5.0,Master of the Flying Guillotine (Du bi quan wa...,Action,4.5,1
3,8477,5.0,"Jetée, La (1962)",Romance|Sci-Fi,4.5,2
4,136850,5.0,Villain (1971),Crime|Drama|Thriller,5.0,1
5,40491,5.0,"Match Factory Girl, The (Tulitikkutehtaan tytt...",Comedy|Drama,5.0,1
6,3567,5.0,Bossa Nova (2000),Comedy|Drama|Romance,5.0,1
7,156605,5.0,Paterson,(no genres listed),4.5,1
8,6835,5.0,Alien Contamination (1980),Action|Horror|Sci-Fi,5.0,1
9,132333,5.0,Seve (2014),Documentary|Drama,5.0,1


In [22]:
movie_r2=pd.merge(movie_r1,links,on='movieId')
movie_r2.drop(['title'],axis=1,inplace=True)
movie_r2

,movieId,Pred_rating,genres,rating,rating_counts,imdbId,tmdbId
0,5746,5.0,Action|Horror|Mystery|Sci-Fi,5.0,1,82431,28893.0
1,5490,5.0,Action|Comedy,5.0,1,74205,19133.0
2,7899,5.0,Action,4.5,1,72913,49636.0
3,8477,5.0,Romance|Sci-Fi,4.5,2,56119,662.0
4,136850,5.0,Crime|Drama|Thriller,5.0,1,67952,30341.0
5,40491,5.0,Comedy|Drama,5.0,1,98532,7974.0
6,3567,5.0,Comedy|Drama|Romance,5.0,1,180837,19600.0
7,156605,5.0,(no genres listed),4.5,1,5247022,370755.0
8,6835,5.0,Action|Horror|Sci-Fi,5.0,1,82000,39916.0
9,132333,5.0,Documentary|Drama,5.0,1,3149640,278990.0


In [23]:
#tags1=tags[['movieId','tag']]
#tags1.groupby('tag')['movieId'].count()
movie_r3=pd.merge(movie_r2,tags[['movieId','tag']],on='movieId',how='left')
movie_r3.head()

,movieId,Pred_rating,genres,rating,rating_counts,imdbId,tmdbId,tag
0,5746,5.0,Action|Horror|Mystery|Sci-Fi,5.0,1,82431,28893.0,NaN
1,5490,5.0,Action|Comedy,5.0,1,74205,19133.0,NaN
2,7899,5.0,Action,4.5,1,72913,49636.0,NaN
3,8477,5.0,Romance|Sci-Fi,4.5,2,56119,662.0,post-apocalyptic
4,136850,5.0,Crime|Drama|Thriller,5.0,1,67952,30341.0,NaN


In [24]:
pd.DataFrame(movie_r3.groupby(['movieId','genres','rating','imdbId'])['tag'].apply(list))

,,,,tag
movieId,genres,rating,imdbId,
3567,Comedy|Drama|Romance,5.0,180837,[nan]
5490,Action|Comedy,5.0,74205,[nan]
5746,Action|Horror|Mystery|Sci-Fi,5.0,82431,[nan]
6835,Action|Horror|Sci-Fi,5.0,82000,[nan]
7899,Action,4.5,72913,[nan]
8477,Romance|Sci-Fi,4.5,56119,[post-apocalyptic]
40491,Comedy|Drama,5.0,98532,[depression]
132333,Documentary|Drama,5.0,3149640,[nan]
136850,Crime|Drama|Thriller,5.0,67952,[nan]


## ------------------------------------------END------------------------------------------------------------------

## APPLYING NEAREST NEIGHBORS TO FIND THE RECOMMECDED MOVIES,ASSIGNMENT IS ABOVE,THIS IS JUST FOR LEARNING PURPOSE

In [25]:
book_data_matrix=csr_matrix(book_pivot.values)
book_data_matrix

<9724x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [26]:
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(book_data_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

## TAKING THE USER_ID AND IN THAT USER_ID, THE MOVIE HE HAS RATED THE MOST,CALCULATING THE DISTANCE WITH REFERENCE TO IT 

In [41]:
user_id=int(input('Enter the user_id:'))
q=book_pivot[user_id][book_pivot[user_id]!=0]
p=q[q==5].reset_index()
query_index=p['movieId'][0]
distance,indices=model_knn.kneighbors(book_pivot.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6404)

Enter the user_id:600


In [42]:
book_pivot.iloc[query_index,:].values.reshape(1,-1)

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 3. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 4. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        4.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 4. , 0. , 0. , 0. , 0. , 2. , 0. , 2. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 4.5, 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 4. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 2. , 0. 

In [43]:
distance

array([[5.55111512e-16, 4.77450676e-01, 4.84724184e-01, ...,
        9.14971423e-01, 9.14971423e-01, 9.15048656e-01]])

In [44]:
indices

array([[1073, 1444,  818, ..., 5985, 3907, 8911]], dtype=int64)

In [45]:
indices.flatten()[1:]

array([1444,  818, 2194, ..., 5985, 3907, 8911], dtype=int64)

In [46]:
print('Recommendatons for {0}\n'.format(book_pivot.index[query_index]))
ls=[]
for i in indices.flatten()[1:]:
    (book_pivot.index[i])
    ls.append(book_pivot.index[i])
se=pd.DataFrame(ls,columns=['movieId'])
movies1=pd.merge(se,movies,on='movieId')
movies_rating=pd.merge(movies1,ratings_mean_count,on='movieId')
movies_rating

Recommendatons for 1394



,movieId,title,genres,rating,rating_counts
0,1968,"Breakfast Club, The (1985)",Comedy|Drama,3.778761,113
1,1079,"Fish Called Wanda, A (1988)",Comedy|Crime,3.922535,71
2,2918,Ferris Bueller's Day Off (1986),Comedy,3.839450,109
3,1265,Groundhog Day (1993),Comedy|Fantasy|Romance,3.944056,143
4,1278,Young Frankenstein (1974),Comedy|Fantasy,3.992754,69
5,1288,This Is Spinal Tap (1984),Comedy,4.015152,66
6,1136,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy,4.161765,136
7,1208,Apocalypse Now (1979),Action|Drama|War,4.219626,107
8,1285,Heathers (1989),Comedy,3.731707,41
9,1222,Full Metal Jacket (1987),Drama|War,4.098039,102


In [51]:
dq={}
for  i in ls:
    pre=recom.predict(user_id,i)
    dq[i]=pre[3]
predict_movies=pd.DataFrame([dq])
predict_movies=predict_movies.T
predict_movies=predict_movies.reset_index()
predict_movies.columns=['movieId','pred_rating']
predict_movies=predict_movies[predict_movies['pred_rating']>4.5]
predict_movies

,movieId,pred_rating
404,633,4.536464
2052,3379,4.849431
2233,3678,4.536464
2400,3951,4.599524
2542,4180,4.559128
2697,4495,4.592525
3084,5328,4.599524
3120,5416,4.599524
3168,5490,5.000000
3425,6201,4.592525


In [52]:
movies1=pd.merge(se,movies,on='movieId')
movies_rating=pd.merge(movies1,ratings_mean_count,on='movieId')
movies_rating_pred=pd.merge(movies_rating,predict_movies,on='movieId')
movies_rating_pred

,movieId,title,genres,rating,rating_counts,pred_rating
0,4180,Reform School Girls (1986),Action|Drama,5.0,1,4.559128
1,3951,Two Family House (2000),Drama,5.0,1,4.599524
2,5328,Rain (2001),Drama|Romance,5.0,1,4.599524
3,5416,Cherish (2002),Comedy|Drama|Thriller,5.0,1,4.599524
4,3678,"Man with the Golden Arm, The (1955)",Drama,5.0,1,4.536464
5,633,Denise Calls Up (1995),Comedy,5.0,1,4.536464
6,4495,Crossing Delancey (1988),Comedy|Romance,5.0,1,4.592525
7,6201,Lady Jane (1986),Drama|Romance,5.0,1,4.592525
8,187717,Won't You Be My Neighbor? (2018),Documentary,5.0,1,4.584888
9,160644,Indignation (2016),Drama,5.0,1,4.584888


## We finded the predicted ratings with counts of movies and also i founded the mean ratings

## Corelation(for learning purpose only)


In [33]:
rating.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [34]:
movie_s=book_pivot1[1]
movie_s.head()

userId
1    4.0
2    0.0
3    0.0
4    0.0
5    4.0
Name: 1, dtype: float64

In [50]:
.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [37]:
ratings_mean_count = pd.DataFrame(rating.groupby('movieId')['rating'].mean())

In [38]:
ratings_mean_count['rating_counts'] = pd.DataFrame(rating.groupby('movieId')['rating'].count())
ratings_mean_count

,rating,rating_counts
movieId,,
1,3.920930,215
2,3.431818,110
3,3.259615,52
4,2.357143,7
5,3.071429,49
6,3.946078,102
7,3.185185,54
8,2.875000,8
9,3.125000,16


In [39]:
rating_count=rating.groupby('movieId')['rating'].count().sort_values(ascending=False).isnull().sum()
rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [40]:
movie_similar=movie_similar.join(ratings_mean_count['rating_counts'])

In [41]:
movie_similar[movie_similar['rating_counts']>50].sort_values('Correlation', ascending=False).head(10)

,Correlation,rating_counts
movieId,,
1,1.000000,215
3114,0.461761,97
1265,0.361540,143
780,0.358473,202
1073,0.357314,119
648,0.352847,162
788,0.350295,82
2355,0.345431,92
364,0.344248,172
